In [1]:
import re
import pandas as pd
from nltk.corpus import stopwords

import copy
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
#from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [2]:
news = pd.read_excel("C:\\Users\\Kratika\\Downloads\\News333.xlsx")
news.head()

,Title,Body,Category,MegaCategory
0,BS-VI transition may lead to dumping of old st...,BS-VI transition may lead to dumping of old st...,Annual Report - Comments made in the Annual Re...,Financial
1,Annual Report 2016-2017 of Bajaj Finserv Limited,It is a broadly described annual report of Baj...,Annual Report - Comments made in the Annual Re...,Financial
2,Annual Report 2017-2018 of Eicher Motors Limit...,It is a broadly described annual report of\nEi...,Annual Report - Comments made in the Annual Re...,Financial
3,Hindalco Industries : Chairman Kumar Mangalam ...,(You can enter multiple email addresses separa...,Annual Report - Comments made in the Annual Re...,Financial
4,"WIPRO continues it's uptrend, although on a we...","WIPRO continues it's uptrend, although on a we...",Annual Report - Comments made in the Annual Re...,Financial


In [3]:
from nltk.tokenize import word_tokenize 

def remove_stop_words(data):
    stop_words = set(stopwords.words('english'))
    stop_words.union(['january','february','march','april','may','june','july','august','september','october','november','december'])
    stop_words.union(['jan','feb','mar','apr','may','jun','jul','aug','sept','oct','nov','dec'])
    stop_words.union(['monday', 'tuesday', 'wednesday', 'thursday','friday','saturday','sunday'])
    stop_words.union(['am', 'pm'])
    word_tokens = word_tokenize(data) 
    filtered_sentence = []
    
    for w in word_tokens: 
        if w.lower() not in stop_words: 
            filtered_sentence.append(w) 
    
    return " ".join(filtered_sentence)

In [4]:
news['text'] = news.Title + " " + news.Body

In [5]:

news['clean_title'] = news['Title'].apply(remove_stop_words)
news['clean_body'] = news['Body'].apply(remove_stop_words)

news['clean_text'] = news['text'].apply(remove_stop_words) 

#remove_stop_words_Body


In [6]:
# ## Split data
print("\nSplitting data")

title_tr, title_te, MegaCategory_tr, MegaCategory_te = train_test_split(news['clean_text'], news.MegaCategory,test_size =.1)
title_tr, title_de, MegaCategory_tr , MegaCategory_de = train_test_split(title_tr,MegaCategory_tr,test_size =.1)


print("Training: ",len(title_tr))
print("Developement: ",len(title_de),)
print("Testing: ",len(title_te))


Splitting data
Training:  2007
Developement:  223
Testing:  248


In [7]:
from sklearn.preprocessing import LabelEncoder

# # Data Preprocessing
# ## Vectorization of data
# Vectorize the data using Bag of words (BOW)
print("\nVectorizing data")
tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
stop_words = nltk.corpus.stopwords.words("english")
vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize, stop_words=stop_words)

vectorizer.fit(iter(title_tr))
Xtr = vectorizer.transform(iter(title_tr))
Xde = vectorizer.transform(iter(title_de))
Xte = vectorizer.transform(iter(title_te))

encoder = LabelEncoder()
encoder.fit(MegaCategory_tr)
Ytr = encoder.transform(MegaCategory_tr)
Yde = encoder.transform(MegaCategory_de)
Yte = encoder.transform(MegaCategory_te)


Vectorizing data


In [8]:
print(Ytr)

[11 11  8 ... 11  5 11]


In [9]:
Xtr

<2007x15905 sparse matrix of type '<class 'numpy.int64'>'
	with 103928 stored elements in Compressed Sparse Row format>

In [10]:
# # Train Models
# ### Baseline Model
# “stratified”: generates predictions by respecting the training set’s class distribution.
print("\n\nTraining baseline classifier")
dc = DummyClassifier(strategy="stratified")
dc.fit(Xtr, Ytr)
dc_pred = dc.predict(Xde)



Training baseline classifier


In [11]:
print(classification_report(Yde, dc_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(dc_pred == Yde)))

                      precision    recall  f1-score   support

Awards & Recognition       0.36      0.38      0.37        79
   Corporate Updates       0.36      0.38      0.37        79
               Event       0.00      0.00      0.00        23
              Events       0.08      0.09      0.08        35
           Financial       0.00      0.00      0.00        23
              Growth       0.36      0.38      0.37        79
       Industry News       0.36      0.38      0.37        79
         Market Buzz       0.08      0.09      0.08        35
       Negative News       0.17      0.17      0.17        24
              People       0.17      0.17      0.17        24
   Stock Performance       0.36      0.38      0.37        79
  Uncategorised News       0.36      0.38      0.37        79

           micro avg       0.28      0.30      0.29     95839
           macro avg       0.20      0.21      0.21     95839
        weighted avg       0.28      0.30      0.29     95839

Accur

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2007, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [12]:

# ### Decision Tree
print("Training Decision tree")
dt = DecisionTreeClassifier()
dt.fit(Xtr, Ytr)
dt_pred = dt.predict(Xde)
print(classification_report(Yde, dt_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(dt_pred == Yde)))

Training Decision tree
                      precision    recall  f1-score   support

Awards & Recognition       0.57      0.73      0.64        79
   Corporate Updates       0.57      0.73      0.64        79
               Event       0.55      0.26      0.35        23
              Events       0.58      0.60      0.59        35
           Financial       0.55      0.26      0.35        23
              Growth       0.57      0.73      0.64        79
       Industry News       0.57      0.73      0.64        79
         Market Buzz       0.58      0.60      0.59        35
       Negative News       0.22      0.29      0.25        24
              People       0.22      0.29      0.25        24
   Stock Performance       0.57      0.73      0.64        79
  Uncategorised News       0.57      0.73      0.64        79

           micro avg       0.55      0.64      0.59     95839
           macro avg       0.49      0.52      0.50     95839
        weighted avg       0.54      0.64    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2007, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [13]:
# ### Random Forest
print("Training Random Forest")
rf = RandomForestClassifier(n_estimators=40)
rf.fit(Xtr, Ytr)
pred = rf.predict(Xde)
print(classification_report(Yde, pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred == Yde)))

Training Random Forest
                      precision    recall  f1-score   support

Awards & Recognition       0.56      0.92      0.70        79
   Corporate Updates       0.56      0.92      0.70        79
               Event       1.00      0.35      0.52        23
              Events       0.66      0.71      0.68        35
           Financial       1.00      0.35      0.52        23
              Growth       0.56      0.92      0.70        79
       Industry News       0.56      0.92      0.70        79
         Market Buzz       0.66      0.71      0.68        35
       Negative News       0.65      0.46      0.54        24
              People       0.65      0.46      0.54        24
   Stock Performance       0.56      0.92      0.70        79
  Uncategorised News       0.56      0.92      0.70        79

           micro avg       0.58      0.80      0.67     95839
           macro avg       0.63      0.65      0.59     95839
        weighted avg       0.60      0.80    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2007, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [14]:

# ### Multinomial Naive Bayesian
print("Training Multinomial Naive Bayesian")
nb = MultinomialNB()
nb.fit(Xtr, Ytr)
pred_nb = nb.predict(Xde)
print(classification_report(Yde, pred_nb, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred_nb == Yde)))

Training Multinomial Naive Bayesian
                      precision    recall  f1-score   support

Awards & Recognition       0.67      0.81      0.73        79
   Corporate Updates       0.67      0.81      0.73        79
               Event       0.65      0.48      0.55        23
              Events       0.59      0.91      0.72        35
           Financial       0.65      0.48      0.55        23
              Growth       0.67      0.81      0.73        79
       Industry News       0.67      0.81      0.73        79
         Market Buzz       0.59      0.91      0.72        35
       Negative News       0.60      0.62      0.61        24
              People       0.60      0.62      0.61        24
   Stock Performance       0.67      0.81      0.73        79
  Uncategorised News       0.67      0.81      0.73        79

           micro avg       0.66      0.77      0.71     95839
           macro avg       0.69      0.69      0.65     95839
        weighted avg       0.67 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2007, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [15]:
# ### Support Vector Classification
print("Training Support Vector Classification")
from sklearn.svm import SVC
svc = SVC()
svc.fit(Xtr, Ytr)
svc_pred = svc.predict(Xde)
print(classification_report(Yde, svc_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(svc_pred == Yde)))

Training Support Vector Classification


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


                      precision    recall  f1-score   support

Awards & Recognition       0.36      0.99      0.53        79
   Corporate Updates       0.36      0.99      0.53        79
               Event       0.00      0.00      0.00        23
              Events       0.62      0.14      0.23        35
           Financial       0.00      0.00      0.00        23
              Growth       0.36      0.99      0.53        79
       Industry News       0.36      0.99      0.53        79
         Market Buzz       0.62      0.14      0.23        35
       Negative News       0.00      0.00      0.00        24
              People       0.00      0.00      0.00        24
   Stock Performance       0.36      0.99      0.53        79
  Uncategorised News       0.36      0.99      0.53        79

           micro avg       0.37      0.71      0.48     95839
           macro avg       0.25      0.44      0.26     95839
        weighted avg       0.32      0.71      0.40     95839

Accur

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2007, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [16]:
# ### Multilayered Perceptron
print("Training Multilayered Perceptron")
mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 20), random_state=1, max_iter=400)
mlp.fit(Xtr, Ytr)
mlp_pred = mlp.predict(Xde)
print(classification_report(Yde, mlp_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(mlp_pred == Yde)))

Training Multilayered Perceptron
                      precision    recall  f1-score   support

Awards & Recognition       0.77      0.85      0.81        79
   Corporate Updates       0.77      0.85      0.81        79
               Event       0.59      0.57      0.58        23
              Events       0.71      0.77      0.74        35
           Financial       0.59      0.57      0.58        23
              Growth       0.77      0.85      0.81        79
       Industry News       0.77      0.85      0.81        79
         Market Buzz       0.71      0.77      0.74        35
       Negative News       0.56      0.62      0.59        24
              People       0.56      0.62      0.59        24
   Stock Performance       0.77      0.85      0.81        79
  Uncategorised News       0.77      0.85      0.81        79

           micro avg       0.74      0.80      0.77     95839
           macro avg       0.71      0.73      0.71     95839
        weighted avg       0.75    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2007, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [17]:
print('Accuracy achieved is ' + str(np.mean(dc_pred   == Yde)))
print('Accuracy achieved is ' + str(np.mean(dt_pred   == Yde)))
print('Accuracy achieved is ' + str(np.mean(pred_nb   == Yde)))
print('Accuracy achieved is ' + str(np.mean(svc_pred  == Yde)))
print('Accuracy achieved is ' + str(np.mean(mlp_pred  == Yde)))

Accuracy achieved is 0.19282511210762332
Accuracy achieved is 0.5022421524663677
Accuracy achieved is 0.6636771300448431
Accuracy achieved is 0.3721973094170404
Accuracy achieved is 0.7085201793721974


In [18]:
# # Final Model: Multilayered Perceptron
# ## Predict test data
print("\n\nPredicting test data using Multilayered Perceptron")
pred_final = mlp.predict(Xte)
print(classification_report(Yte, pred_final, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(accuracy_score(Yte,pred_final)))



Predicting test data using Multilayered Perceptron
                      precision    recall  f1-score   support

Awards & Recognition       0.73      0.86      0.79        95
   Corporate Updates       0.73      0.86      0.79        95
               Event       0.50      0.42      0.46        26
              Events       0.76      0.66      0.70        38
           Financial       0.50      0.42      0.46        26
              Growth       0.73      0.86      0.79        95
       Industry News       0.73      0.86      0.79        95
         Market Buzz       0.76      0.66      0.70        38
       Negative News       0.58      0.58      0.58        26
              People       0.58      0.58      0.58        26
   Stock Performance       0.73      0.86      0.79        95
  Uncategorised News       0.73      0.86      0.79        95

           micro avg       0.71      0.78      0.74    112183
           macro avg       0.65      0.67      0.65    112183
        weighte

In [19]:
output = {"title":vectorizer.inverse_transform(Xte), "predicted":encoder.inverse_transform(pred_final),"actual": encoder.inverse_transform(Yte)}

In [20]:
#df = pd.DataFrame(output, columns=["title","predicted","actual"])
#df.to_csv("C:\\Users\\Kratika\\Downloads\\News333_title_predication_1.csv")

In [21]:
pred_final = mlp.predict(Xte)
print('Accuracy achieved is ' + str(np.mean(pred_final == Yte)))
output = {"text":vectorizer.inverse_transform(Xte), "predicted":encoder.inverse_transform(pred_final),"actual": encoder.inverse_transform(Yte)}
df = pd.DataFrame(output, columns=["text","predicted","actual"])


Accuracy achieved is 0.6612903225806451


In [23]:
#df = pd.DataFrame(output, columns=["text","predicted","actual"])
#df.to_csv("C:\\Users\\Kratika\\Downloads\\News333_title_body_predication.csv")